In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a

In [123]:
def get_articles_adress(page):
    '''
    Возвращает список адресов всех статей
    '''
    articles_adress = []
    id_str = '/item.asp?id='
    for i in page.findAll('a'):
        try:
            if id_str in i['href']:
                num_id = i['href']
                articles_adress.append('https://elibrary.ru' + num_id)
        except:
            continue
    return articles_adress

def find_data(adress):
    '''
    Возвращает list из [названия статьи, авторы, число цитирований, абстракт]
    '''
    assert isinstance(adress, str)
    sleep(np.random.rand() * 2)
    art = requests.get(adress)
    soup = BeautifulSoup(art.content.decode('utf-8'), "html.parser")
    data = []
    # Добавление Автора и Названия статьи
    for i in soup.findAll('input'):
        try:    
            if not i['value'][0].isdigit() and (i['value'][0] == i['value'][0].upper()):
                data.append(i['value'])
        except:
            continue
    
    if len(data) < 2:
        data.insert(0, 'Incognito')
    # Добавление количества цитирований
    try:
        quot = soup.findAll('a', attrs= {'href':['cit_items.asp?id=' + adress.split('=')[-1]]})[0].text
    except:
        quot = '0'
    data.append(quot)
    # Добавление абстракта
    abstract = soup.find('p').text
    key = 'https://doi'
    if abstract:
        data.append(abstract)
    else:
        for i in soup.findAll('a'):
            try:
                if key in i['href']:
                    adress_r = i['href']
                    break
            except:
                continue
        abstract_journal = BeautifulSoup(requests.get(adress_r).text, "html.parser").find('p').text
        data.append(abstract_journal) #если нет абстракта берет его с сайта журнала
    return data

def parse_journal(item):
    name, web_adress =  item[0], item[1]
    print(name, web_adress)
    columns = ['Authors', 'Title', 'Qoutes', 'Abstract', 'Link']
    articles = pd.DataFrame(columns=columns)
    xp_main = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
    browser = Chrome(executable_path="./chromedriver")                                       # ./geckodriver тоже вроде можно
    wait = WebDriverWait(browser, 300, 1)
    browser.get(str(web_adress))                                                             # Заходим на первую страницу журнала
    for i in range(50):
        search_form = wait.until(EC.visibility_of_element_located((By.XPATH, xp_main)))
        page = BeautifulSoup(browser.page_source, "html.parser")                             # Забираем страничку и преобразуем ее в BS
        new_id = get_articles_adress(page)                                                   # Собираем со странички все адреса статей
        for i in range(len(new_id)):
            if i % 3 == 0:
                sleep(10)
            d = find_data(new_id[i])
            d.append(new_id[i])                                                                    # Хранится в виде: название статьи: авторы, цитирования, абстракт, ссылка 
            print(len(d), i)
            sleep(8 * np.random.rand())
            articles = articles.append(pd.Series(d, index=columns), ignore_index=True)
            articles.to_csv(name, sep="@")
        search_form.click()                                                                  # Жмякаем на кнопку
        sleep(5)
        

/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a


In [120]:
journals = {
    'NATURE_PHYSICS'    : 'https://elibrary.ru/title_items.asp?id=25368',
    'PHYSICAL_REVIEW_B' : "https://elibrary.ru/title_items.asp?id=21814",
    'PHYSICAL_REVIEW_C' : "https://elibrary.ru/title_items.asp?id=21815",
    'PHYSICAL_REVIEW_LETTERS' : "https://elibrary.ru/title_items.asp?id=21820",
    'JOURNAL_OF_EXPERIMENTAL_AND_THEORETICAL_PHYSICS' : 'https://elibrary.ru/title_items.asp?id=7467',
    'JOURNAL_OF_EXPERIMENTAL_AND_THEORETICAL_PHYSICS_LETTERS_(JETP_LETTERS)' : 'https://elibrary.ru/title_items.asp?id=7468'    
}

In [121]:
j=list(journals.items())
j[-1]

('JOURNAL_OF_EXPERIMENTAL_AND_THEORETICAL_PHYSICS_LETTERS_(JETP_LETTERS)',
 'https://elibrary.ru/title_items.asp?id=7468')

In [122]:
parse_journal(j[0])

NATURE_PHYSICS https://elibrary.ru/title_items.asp?id=25368


AttributeError: 'NoneType' object has no attribute 'text'

In [102]:
df = pd.read_csv(j[0][0],sep="@")

In [103]:
df

,Unnamed: 0,Authors,Title,Qoutes,Abstract,Link


In [72]:
p = requests.get('https://elibrary.ru/item.asp?id=36128600')

In [73]:
p = BeautifulSoup(p.content.decode('utf-8'), "html.parser")

In [76]:
h = []
for i in p.findAll('input'):
    try:    
        if not i['value'][0].isdigit() and (i['value'][0] == i['value'][0].upper()):
            h.append(i['value'])
    except:
        continue


In [77]:
h

['Incognito', 'Topology reaches higher spheres']

In [93]:
https = "https://elibrary.ru"
artical_data = {}
artical_adresses = []
xp_main    = '/html/body/div[3]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'
xp_reserve = '/html/body/div[2]/table/tbody/tr/td/table[1]/tbody/tr/td[2]/form/table/tbody/tr[2]/td[1]/table/tbody/tr/td/div[6]/table/tbody/tr/td[13]/a'

browser = Firefox(executable_path="./geckodriver")     # ./geckodriver тоже вроде можно
wait = WebDriverWait(browser, 30, 1)
browser.get('https://elibrary.ru/title_items.asp?id=21814')                          # Заходим на первую страницу журнала
for i in range(4):
    search_form = wait.until(EC.visibility_of_element_located((By.XPATH, xp_main)))
    page = BeautifulSoup(browser.page_source, "html.parser")                             # Забираем страничку и преобразуем ее в BS
    new_id = get_articles_adress(page)                                                   # Собираем со странички все адреса статей
    for _id in new_id:
        d = find_data(_id)
        artical_data[d[1]] = [d[0],d[2],d[3],_id]                                        # Хранится в виде: название статьи: авторы, цитирования, абстракт, ссылка 
        sleep(np.random.rand() * 2)
    search_form.click()                                                                  # Жмякаем на кнопку
    sleep(5)

WebDriverException: Message: TypeError: this.curBrowser.contentBrowser is null


In [32]:
def get_proxy():
    return proxies_list[np.random.randint(0, len(proxies_list))]
proxies_list = [{'http':'http://'+i} for i in ['67.149.217.254:10200',
                '64.20.74.24:45554','62.37.237.101:8080',
                '180.234.206.77:8080',
                '78.11.85.13:8080','109.188.81.101:8080',
                '139.59.17.113:8080','191.179.147.46:11421',
                '111.68.99.42:8080','80.241.219.66:3128',
                '201.20.94.106:8080','216.229.120.173:45554',
                '116.58.247.31:3128','103.9.115.142:3128',
                '82.164.99.193:10200','80.188.79.138:8080',
                '36.75.113.224:8080',
                '1.20.204.163:8080','97.77.49.151:45554',
                '178.54.44.24:8080',
                '65.182.136.153:45554', '111.76.129.223:808',
                 '203.142.81.205'+':'+'8080', 
                '42.202.35.185'+':'+'8118', '189.16.249.114'+':'+'8080',
                '66.162.122.24'+':'+'8080']]